In [1]:
from fuzzy import train, setup
from db import process
from file import load, append

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as ws
from IPython.display import display
from matplotlib.patches import Rectangle

In [3]:
def get_color():
    return np.random.randint(0, 256, 3) / 255

def get_intense(r, g, b):
    return 0.2126 * r + 0.7152 * g + 0.0722 * b

In [4]:
col = None
drop = ws.Dropdown(options=[("Тёмный", "d"), ("Средний.", "m"), ("Светлый", "b")], description="Тип:")
btn1 = ws.Button(description="Сгенерировать цвет")
btn2 = ws.Button(description="Подтвердить")
out = ws.Output()

In [5]:
def gen_click(btn):
    with out:
        global col
        out.clear_output()
        col = get_color()
        plt.gca().add_patch(Rectangle((1,1), 5, 5, fill=True, color=col))
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
        plt.axis("scaled")
        plt.show()

def send_click(btn):
    with out:
        global col
        append({"intensity": np.round(get_intense(*col), 2), "mark": drop.value}, "data/db.csv")
        col = None
        out.clear_output()


btn1.on_click(gen_click)
btn2.on_click(send_click)

In [6]:
display(btn1, out, drop, btn2)

Button(description='Сгенерировать цвет', style=ButtonStyle())

Output()

Dropdown(description='Тип:', options=(('Тёмный', 'd'), ('Средний.', 'm'), ('Светлый', 'b')), value='d')

Button(description='Подтвердить', style=ButtonStyle())

In [7]:
def sigmoid(x, k, a):
    return 1 / (1 + np.exp(-k * (-x - a)))

def gauss(x, k, a):
    return np.exp(-k * (x - a))

is_dark, is_medium, is_bright = None, None, None

In [8]:
out2 = ws.Output()
def train_click(btn):
    with out2:
        out2.clear_output()
        global is_dark, is_medium, is_bright
        db = load("data/db.csv")
        data = process(db, db.columns[1], db.columns[0])
        funcs = train(data, "intensity", (sigmoid, gauss, sigmoid), ("b", "m", "d"))
        is_dark = setup(sigmoid, funcs["d"], ("k", "a"))
        is_medium = setup(gauss, funcs["m"], ("k", "a"))
        is_bright = setup(sigmoid, funcs["b"], ("k", "a"))
        print("Обучение завершено!!")
    

btn3 = ws.Button(description="Обучить")
btn3.on_click(train_click)
display(btn3, out2)

Button(description='Обучить', style=ButtonStyle())

Output()

In [9]:
out3 = ws.Output()

def predict_click(btn):
    with out3:
        global col
        out3.clear_output()
        col = get_color()
        plt.gca().add_patch(Rectangle((1,1), 5, 5, fill=True, color=col))
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
        plt.axis("scaled")
        plt.show()
        values = tuple(f(np.round(get_intense(*col), 2)) for f in (is_dark, is_bright, is_medium))
        idx = max(range(len(values)), key=values.__getitem__)
        print(values)
        print("Dark" if idx == 0 else "Light" if idx == 1 else "Medium")

btn4 = ws.Button(description="Проверить")
btn4.on_click(predict_click)
display(btn4, out3)


Button(description='Проверить', style=ButtonStyle())

Output()